In [1]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from acoustic_BEM.geometry import Body, Field
from acoustic_BEM.mesh import Mesh
from acoustic_BEM.elements import ContinuousP1Mesh, DiscontinuousP1Mesh
from acoustic_BEM.integrators import ElementIntegratorCollocation
from acoustic_BEM.matrix_assembly import ContinuousAssembler, DiscontinuousAssembler
from acoustic_BEM.solve import BEMSolver

import cProfile
import pstats

%load_ext autoreload
%autoreload 2

In [2]:
def icosahedron():
    """Generate vertices and faces of a unit icosahedron."""
    t = (1.0 + np.sqrt(5.0)) / 2.0
    V = np.array([
        [-1,  t, 0], [ 1,  t, 0], [-1, -t, 0], [ 1, -t, 0],
        [ 0, -1,  t], [ 0,  1,  t], [ 0, -1, -t], [ 0,  1, -t],
        [ t,  0, -1], [ t,  0,  1], [-t,  0, -1], [-t,  0,  1],
    ], dtype=float)
    V /= np.linalg.norm(V, axis=1)[:, None]
    
    F = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7,10], [0,10,11],
        [1, 5, 9], [5,11, 4], [11,10,2], [10,7,6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4,11], [6, 2,10], [8, 6, 7], [9, 8, 1]
    ], dtype=int)
    return V, F

def subdivide_sphere(V, F, levels=1):
    """Loop-subdivide each triangle and re-project to unit sphere."""
    for _ in range(levels):
        mid_cache = {}
        newF = []
        
        def midpoint(i, j):
            key = tuple(sorted((i, j)))
            if key in mid_cache:
                return mid_cache[key]
            m = (V[i] + V[j]) * 0.5
            m = m / np.linalg.norm(m)
            idx = len(V_list)
            V_list.append(m)
            mid_cache[key] = idx
            return idx
        
        V_list = [v.copy() for v in V]
        for a, b, c in F:
            ab = midpoint(a, b)
            bc = midpoint(b, c)
            ca = midpoint(c, a)
            newF += [
                [a, ab, ca], [b, bc, ab],
                [c, ca, bc], [ab, bc, ca],
            ]
        V = np.asarray(V_list)
        F = np.asarray(newF, dtype=int)
    return V, F

In [3]:
radius = 1  # sphere radius [m]
rho0 = 1.225    # air density [kg/m³]
c0 = 343.0      # speed of sound [m/s]
Vr = 1.0        # radial velocity [m/s]

# Mesh generation
subdiv = 2  # subdivision level (3 is good balance, 4 for finer mesh)
nodes, elements = icosahedron()
nodes, elements = subdivide_sphere(nodes, elements, levels=subdiv)
nodes *= radius

print(f"Mesh: {nodes.shape[0]} nodes, {elements.shape[0]} elements")

Mesh: 162 nodes, 320 elements


In [4]:
# Field evaluation point
# field_distance = 2.0

field_pts = np.array([[0, 0, 2.0],
                      [0, 0, 2.5],
                      [0, 0, 3.0],
                      [0, 2.0, 0],
                      [0, 2.5, 0],
                      [0, 3.0, 0]])

# Frequency sweep
freq_min, freq_max, n_freq = 50, 500, 50
frequencies = np.linspace(freq_min, freq_max, n_freq)

field = Field(
    # field_extent=np.array([[field_distance, field_distance+0.1],
    #                       [0, 0.1], [0, 0.1]]),
    # num_points=np.array([1, 1, 1]),
    rho0=rho0,
    c0=c0,
)

In [5]:
f = 200  # example frequency [Hz]
omega = 2 * np.pi * f
k = omega / c0

# Boundary condition (Neumann - velocity BC)
vel_bc = np.ones(nodes.shape[0]) * Vr
neumann_bc = vel_bc * 1j * omega * rho0

# Create Body and base Mesh
sphere = Body(
    mesh_nodes=nodes, 
    mesh_elements=elements, 
    Neumann_BC=neumann_bc,
    Dirichlet_BC=None,
    frequency=f,
)

base_mesh = Mesh(
    source_object=sphere,
    peripheral_objects=None,
    field=field
)

integrator = ElementIntegratorCollocation(k=k)    

disc_mesh_int = DiscontinuousP1Mesh(
    base_mesh, 
    collocation_strategy="interior_shifted",
    shift_factor=0.15
)
disc_assembler_int = DiscontinuousAssembler(disc_mesh_int, integrator, 
                                            quad_order=7)
disc_solver_int = BEMSolver(disc_assembler_int)

profiler = cProfile.Profile()
profiler.enable()
S = disc_assembler_int.assemble("S", verbose=False)
profiler.disable()
stats = pstats.Stats(profiler)
stats.sort_stats('cumulative')
stats.print_stats(20)

Pre-computing Telles quadrature cache...
Cached 31140 Telles quadrature rules.
         3695952 function calls (3695904 primitive calls) in 13.519 seconds

   Ordered by: cumulative time
   List reduced from 349 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   13.519    6.759 d:\Kristof Cufar\Acoustic_BEM\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3663(run_code)
      8/2    0.000    0.000   13.519    6.759 {built-in method builtins.exec}
        1    0.000    0.000   13.519   13.519 d:\Kristof Cufar\Acoustic_BEM\acoustic_BEM\matrix_assembly.py:423(assemble)
        1    2.625    2.625   13.519   13.519 d:\Kristof Cufar\Acoustic_BEM\acoustic_BEM\matrix_assembly.py:449(_assemble_interior_shifted)
    31140    1.121    0.000    5.362    0.000 d:\Kristof Cufar\Acoustic_BEM\acoustic_BEM\quadrature.py:69(map_to_physical_triangle_batch)
    32100    0.056    0.000    4.173    0.000 d:\Kristof Cu